**Work 5**

In [1]:
import json
import os

# Installing the Kaggle package
!pip install kaggle 

!mkdir /root/.kaggle/ 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"avielbbaz","key":"b9e35e8b71e0611fbd90b49b45fde304"}


# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!kaggle datasets list -s Blog
!mkdir ./datasets
!mkdir ./datasets/Blog-Data
!mkdir ./datasets/UFO

ref                                                         title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rtatman/blog-authorship-corpus                              Blog Authorship Corpus                           290MB  2017-08-15 21:20:54           9448        125  0.8235294        
kaggle/kaggle-blog-winners-posts                            Kaggle Blog: Winners' Posts                      519KB  2016-09-21 02:21:21            786         44  0.7058824        
lakritidis/identifying-influential-bloggers-techcrunch      Identifying Influential Bloggers: Techcrunch     112MB  2020-03-30 19:22:09           1076         44  1.0              
saurabhbagchi/sample-blog-corpus                            Sample Blog Corpus                 

In [ ]:
!kaggle datasets list -s UFO

ref                                                            title                                     size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  ---------  ---------------  
NUFORC/ufo-sightings                                           UFO Sightings                             10MB  2019-11-13 19:45:57          31532        588  0.5882353        
camnugent/ufo-sightings-around-the-world                       UFO Sightings around the world             5MB  2017-08-15 15:46:17           3588         75  0.8235294        
rishidamarla/ufo-sightings-approx-100000                       UFO Sightings (Approx. 90,000)            38MB  2021-05-15 03:12:43            607         34  0.9411765        
mysarahmadbhat/ufo-sightings                                   UFO Sightings                              5MB  2021-08-0

In [3]:
!kaggle datasets download rtatman/blog-authorship-corpus -p ./datasets/Blog-Data
!unzip ./datasets/Blog-Data/*.zip -d ./datasets/Blog-Data
!ls ./datasets/Blog-Data

 97% 281M/290M [00:02<00:00, 120MB/s]
100% 290M/290M [00:02<00:00, 131MB/s]
Archive:  ./datasets/Blog-Data/blog-authorship-corpus.zip
  inflating: ./datasets/Blog-Data/blogtext.csv  
blog-authorship-corpus.zip  blogtext.csv


In [4]:
!kaggle datasets download -d NUFORC/ufo-sightings  -p ./datasets/UFO
!unzip ./datasets/UFO/*.zip  -d ./datasets/UFO
!ls ./datasets/UFO


 49% 5.00M/10.2M [00:00<00:00, 47.7MB/s]
100% 10.2M/10.2M [00:00<00:00, 83.9MB/s]
Archive:  ./datasets/UFO/ufo-sightings.zip
  inflating: ./datasets/UFO/complete.csv  
  inflating: ./datasets/UFO/scrubbed.csv  
complete.csv  scrubbed.csv  ufo-sightings.zip


In [5]:
import pandas as pd
import numpy as np
import re

Part 1 - Blog Authorship Corpus

In [6]:
blog_text_df = pd.read_csv('/content/datasets/Blog-Data/blogtext.csv')
blog_text_df = blog_text_df.drop_duplicates()
blog_text_df = blog_text_df.dropna()
blog_text_df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ..."
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i..."
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin..."
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as..."


Q1 

In [7]:
r = re.compile('(\w{8,})')
new_blog_df = blog_text_df
new_blog_df['sign_8_or_more_char'] = new_blog_df['sign'].apply(lambda x: r.findall(x))
signs = []
for sign in new_blog_df['sign_8_or_more_char']:
  if len(sign) != 0:
    signs.append(sign[0])
sign_8_or_more_char = set(signs)
sign_8_or_more_char

{'Aquarius', 'Capricorn', 'Sagittarius'}

Q2 

In [8]:
from datetime import datetime as dt
blog_text_df['date'] = pd.to_datetime(blog_text_df['date'],errors='coerce')
blog_with_year_df = blog_text_df
blog_with_year_df['year'] = blog_with_year_df['date'].dt.strftime('%Y')
group_by_year = blog_with_year_df.groupby('year',as_index = False).count()
group_by_year = group_by_year[['year','text']] 
group_by_year.sort_values('text',ascending=False)

,year,text
5,2004,537544
4,2003,102467
3,2002,22294
2,2001,5581
1,2000,894
0,1999,70
7,2006,23
6,2005,6


Q3

In [9]:
r2 = re.compile('([a-zA-Z0-9_\-\.]+@[a-zA-Z0-9_\-\.]+\.[a-zA-Z]{2,5})')
blog_text_df['email'] = blog_text_df['text'].apply(lambda x: len(r2.findall(x)))

In [10]:
g_by_topic = blog_text_df.groupby('topic',as_index=False).sum()
g2 = g_by_topic.sort_values(['email'],ascending=False).reset_index(drop=True)
print("The number of valid emails for each category in the blog is:")
display(g2[['topic','email']])

The number of valid emails for each category in the blog is:


,topic,email
0,indUnk,1989
1,Student,1110
2,Non-Profit,468
3,Internet,389
4,Education,339
5,Communications-Media,327
6,Technology,287
7,Arts,264
8,Publishing,84
9,Engineering,83


Q4

In [11]:
blogger_with_most_emails_df =  blog_text_df[blog_text_df['email']==blog_text_df['email'].max()]
blogger_with_most_emails_df = blogger_with_most_emails_df[['id','email']].reset_index(drop=True)
blogger_with_most_emails_df

,id,email
0,1383289,93


Q5

In [12]:
r_long_num = re.compile('([\d,]+)')
def max_length(list1):
  max_d = ''
  for i in list1:
    if len(str(i)) >= len(max_d):
      max_d = i
  return len(max_d)
blog_text_df['len_of_longest_num'] = blog_text_df['text'].apply(lambda x: max_length(r_long_num.findall(x)))
blog_topic_most_digits = blog_text_df[blog_text_df['len_of_longest_num']==blog_text_df['len_of_longest_num'].max()]
blog_topic_most_digits[['id','len_of_longest_num']].reset_index(drop=True)

,id,len_of_longest_num
0,3301387,193


Part 2 - UFO Sightlights

In [13]:
ufo_scrubbed_df = pd.read_csv('/content/datasets/UFO/scrubbed.csv',low_memory=False)
ufo_scrubbed_df.drop_duplicates()
ufo_scrubbed_df.dropna()
ufo_scrubbed_df

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013 21:15,nashville,tn,us,light,600,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.1658333,-86.784444
80328,9/9/2013 22:00,boise,id,us,circle,1200,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.6136111,-116.202500
80329,9/9/2013 22:00,napa,ca,us,other,1200,hour,Napa UFO&#44,9/30/2013,38.2972222,-122.284444
80330,9/9/2013 22:20,vienna,va,us,circle,5,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.9011111,-77.265556


Q1

In [14]:
ufo_scrubbed_df['datetime'] = pd.to_datetime(ufo_scrubbed_df['datetime'],errors = 'coerce')
ufo_scrubbed_df['year'] = ufo_scrubbed_df['datetime'].dt.strftime('%Y')
def count_ufo(d_f,country,year):
  return (d_f.loc[(d_f['year']==year) & (d_f['country']== country),['year','country']].count().values[0])
print(f"for example , the number of ufo sights in the us at 1949 is:{count_ufo(ufo_scrubbed_df,'us','1949')}")

for example , the number of ufo sights in the us at 1949 is:15


Q2

In [15]:
ufo_scrubbed_df['month'] = ufo_scrubbed_df['datetime'].dt.strftime('%B')
g_by_month = ufo_scrubbed_df.groupby('month',as_index=False).size()
g_by_month = g_by_month.rename(columns={'size':'number_of_sights'})
month_of_most_sights = g_by_month[g_by_month['number_of_sights']==g_by_month['number_of_sights'].max()] 
print("the month with most ufo sights is:")
display(month_of_most_sights)

the month with most ufo sights is:


,month,number_of_sights
5,July,9441


Q3

In [16]:
g_by_state = ufo_scrubbed_df.groupby('state',as_index=False).size()
g_by_state = g_by_state.rename(columns={'size':'number_of_sights'})
g_by_state = g_by_state.sort_values('number_of_sights',ascending=False)
display(g_by_state[['state']].reset_index(drop=True))

,state
0,ca
1,wa
2,fl
3,tx
4,ny
...,...
62,nf
63,nt
64,pe
65,yt


Q4

In [17]:
r_word_couple = re.compile('(\w{7,}\s+\w{7,})')
ufo_scrubbed_df['couple_words'] = ufo_scrubbed_df['comments'].apply(lambda x: len(r_word_couple.findall(str(x))))
most_couple_word_comment = ufo_scrubbed_df[ufo_scrubbed_df['couple_words']==ufo_scrubbed_df['couple_words'].max()]
display(most_couple_word_comment[['comments']].reset_index(drop=True))

,comments
0,Glowing&#44spinning rapidly&#44displaying quic...


Q5

In [18]:
r_6_digits = re.compile('(d{6}|\d{3},?\d{3})')
ufo_scrubbed_df['most_6_digit_text'] = ufo_scrubbed_df['comments'].apply(lambda x: len(r_6_digits.findall(str(x))))
g_by_country = ufo_scrubbed_df.groupby('country',as_index=False).sum()
country_with_most_6_digits = g_by_country[g_by_country['most_6_digit_text']==g_by_country['most_6_digit_text'].max()]
country_with_most_6_digits[['country','most_6_digit_text']].reset_index(drop=True)

,country,most_6_digit_text
0,us,97


Q6

In [19]:
r_final = re.compile('(\w{2}\d{1}\w{1})')
ufo_scrubbed_df['comment_format'] = ufo_scrubbed_df['comments'].apply(lambda x: len(r_final.findall(str(x))))
ufo_1 = ufo_scrubbed_df.loc[ufo_scrubbed_df['comment_format']>0]
ufo_1.drop(['year','month','couple_words','most_6_digit_text'], axis=1, inplace=True)
ufo_1

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,comment_format
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,1
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,1
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,2
8,1966-10-10 20:00:00,pell city,al,us,disk,180,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.5861111,-86.286111,1
15,1972-10-10 19:00:00,harlan county,ky,us,circle,1200,20minutes,On october 10&#44 1972 myself&#44my 5yrs.daugh...,9/15/2005,36.8430556,-83.321944,1
...,...,...,...,...,...,...,...,...,...,...,...,...
80219,2007-09-09 21:30:00,antelope,ca,us,other,900,15 +- mins,strange object over north sacramento 9:30ish p...,10/8/2007,38.7083333,-121.328889,1
80241,2009-09-09 20:10:00,haverhill,ma,us,light,120,approx 2 min,On 09/09/2009 at 20:00 ET I notice what appear...,12/12/2009,42.7761111,-71.077778,1
80266,2010-09-09 20:30:00,tinley park,il,us,light,300,5 mins,I was outside BBQ and as always I tend to look...,11/21/2010,41.5733333,-87.784444,1
80286,2011-09-09 23:00:00,kenmore,wa,us,changing,5400,1.5 hours,UFO changing colors&#44 shapes and pulsating -...,10/10/2011,47.7575000,-122.242778,1
